In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time

In [2]:
from LagsCreator import LagsCreator

In [3]:
data = np.array([[np.nan, 2, np.nan, np.nan, np.nan, np.nan, np.nan, 4, np.nan, np.nan, np.nan, np.nan, np.nan, 6, np.nan, 2, np.nan, np.nan, np.nan, np.nan, np.nan, 4, np.nan, np.nan, np.nan, np.nan, np.nan, 6], 
                 [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 
                 [3]*28])

In [4]:
df = pd.DataFrame(np.transpose(data), index = pd.date_range("2018-08-22", periods = data.shape[1]), columns = pd.MultiIndex.from_product([["Group 1"], ["A", "B", "C"]], names = ["Group", "Time-series"]))

In [5]:
df["Group 1"]

Time-series,A,B,C
2018-08-22,NaN,NaN,3.0
2018-08-23,2.0,NaN,3.0
2018-08-24,NaN,NaN,3.0
2018-08-25,NaN,NaN,3.0
2018-08-26,NaN,NaN,3.0
2018-08-27,NaN,NaN,3.0
2018-08-28,NaN,NaN,3.0
2018-08-29,4.0,NaN,3.0
2018-08-30,NaN,NaN,3.0
2018-08-31,NaN,NaN,3.0


In [6]:
lags_dict = dict()
# Define lags for each indicator.
lags_dict["A"] = np.array([1, 3])
lags_dict["B"] = np.array([1, 2, 3, 5])
lags_dict["C"] = 0

In [7]:
t1=time.time()
creator = LagsCreator(df["Group 1"], lags_dict, "B", return_dataframe = True,
                      feature_time = ["Day", "Month", "Year", "Dayofweek"]) 
t2=time.time()
print(t2-t1)

0.022887706756591797


In [8]:
t1=time.time()
X_train, y_train, X_test = creator.to_supervised(h = 2, step = 1, single_step = True)
t2=time.time()
print(t2-t1)

0.003988742828369141


In [9]:
X_train

Features      A           B                     C Day Month  Year Dayofweek
Lags     x(t-2) x(t) x(t-4) x(t-2) x(t-1) x(t)  x   x     x     x         x
0             4    2      1      3      4    5  3  11     9  2018         1
1             4    2      2      4      5    6  3  12     9  2018         2
2             4    2      3      5      6    7  3  13     9  2018         3
3             6    4      4      6      7    8  3  14     9  2018         4
4             6    4      5      7      8    9  3  15     9  2018         5
5             6    4      6      8      9   10  3  16     9  2018         6
6             6    4      7      9     10   11  3  17     9  2018         0
7             6    4      8     10     11   12  3  18     9  2018         1

In [10]:
y_train

Target,B
Prediction horizon,x(t+2)
0,7
1,8
2,9
3,10
4,11
5,12
6,13
7,14


In [11]:
X_test

Features      A           B                     C Day Month  Year Dayofweek
Lags     x(t-2) x(t) x(t-4) x(t-2) x(t-1) x(t)  x   x     x     x         x
0             2    6     10     12     13   14  3  20     9  2018         3

In [12]:
train, test = creator.visualization(boundaries = False, gif = True)

In [13]:
train[2]

Time-series,A,B,C
2018-08-22,nan,nan,3.0
2018-08-23,2.0,nan,3.0
2018-08-24,nan,nan,3.0
2018-08-25,nan,nan,3.0
2018-08-26,nan,nan,3.0
2018-08-27,nan,nan,3.0
2018-08-28,nan,nan,3.0
2018-08-29,4.0,nan,3.0
2018-08-30,nan,nan,3.0
2018-08-31,nan,nan,3.0
